In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyarrow as pa
import pyarrow.parquet as pq
import yaml
from datetime import date
from dateutil.relativedelta import relativedelta

In [ ]:
config_file = 'config.yml'
with open(config_file, 'r') as ymlfile:
    cfg = yaml.safe_load(ymlfile)

# insert your API key here
API_KEY = cfg['api_glassnode']['api_glassnode_key']

In [ ]:
# Hash Rate
res = requests.get('https://api.glassnode.com/v1/metrics/mining/hash_rate_mean',
    params={'a': 'BTC', 'api_key': API_KEY})

# convert to pandas dataframe
list_hash_rate = list(eval(res.text))
t = [d['t'] for d in list_hash_rate]
v = [d['v'] for d in list_hash_rate]
df_hash_rate = pd.DataFrame(columns=[
        't', 
        'v'])

df_hash_rate['t'] = t
df_hash_rate['v'] = v
df_hash_rate['t'] = pd.to_datetime(df_hash_rate['t'], unit='s')
df_hash_rate['v'] = df_hash_rate['v'].astype('float')

plt.plot(df_hash_rate['t'], df_hash_rate['v'])

In [ ]:
# SOPR
res = requests.get('https://api.glassnode.com/v1/metrics/indicators/sopr',
    params={'a': 'BTC', 'api_key': API_KEY})

# convert to pandas dataframe
df_sopr = pd.read_json(res.text, convert_dates=['t'])
plt.plot(df_sopr['t'], df_sopr['v'])

In [ ]:
# Active addresses
res = requests.get('https://api.glassnode.com/v1/metrics/addresses/active_count',
    params={'a': 'BTC', 'api_key': API_KEY})

# convert to pandas dataframe
df_active = pd.read_json(res.text, convert_dates=['t'])
plt.plot(df_active['t'], df_active['v'])

In [ ]:
# transactions count
res = requests.get('https://api.glassnode.com/v1/metrics/transactions/count',
    params={'a': 'BTC', 'api_key': API_KEY})

# convert to pandas dataframe
df_transactions = pd.read_json(res.text, convert_dates=['t'])
plt.plot(df_transactions['t'], df_transactions['v'])

In [ ]:
#mining/difficulty_latest
res = requests.get('https://api.glassnode.com/v1/metrics/mining/difficulty_latest',
    params={'a': 'BTC', 'api_key': API_KEY})

# convert to pandas dataframe
list_difficulty = list(eval(res.text))
t = [d['t'] for d in list_difficulty]
v = [d['v'] for d in list_difficulty]
df_difficulty = pd.DataFrame(columns=[
        't', 
        'v'])

df_difficulty['t'] = t
df_difficulty['v'] = v
df_difficulty['t'] = pd.to_datetime(df_difficulty['t'], unit='s')
df_difficulty['v'] = df_difficulty['v'].astype('float')

plt.plot(df_difficulty['t'], df_difficulty['v'])

In [ ]:
# stock_to_flow_ratio
import plotly.graph_objects as go

res = requests.get('https://api.glassnode.com/v1/metrics/indicators/stock_to_flow_ratio',
    params={'a': 'BTC', 'api_key': API_KEY})

# convert to pandas dataframe
df_s2f = pd.read_json(res.text, convert_dates=['t'])
df_s2f['daysTillHalving'] = [d.get('daysTillHalving') for d in df_s2f['o']]
df_s2f['ratio'] = [d.get('ratio') for d in df_s2f['o']]
df_s2f.drop('o', inplace=True, axis=1)
df_s2f.set_index('t', inplace=True)

# price
res = requests.get('https://api.glassnode.com/v1/metrics/market/price_usd_close',
    params={'a': 'BTC', 'api_key': API_KEY})

# convert to pandas dataframe
df_price = pd.read_json(res.text, convert_dates=['t'])
df_price.set_index('t', inplace=True)
df_price.rename(columns = {'v':'ClosePrice'}, inplace = True)
result = pd.merge(df_s2f, df_price, how='left', on='t')
#result = result['2017-01-01':'2022-12-31']

fig = go.Figure()
fig.add_trace(go.Scatter(x=result.index, 
                         y=result['ClosePrice'],
                         name='Price',
                         mode='markers',
                         marker=dict(
                            size=6,
                            color=result['daysTillHalving'], #set color equal to a variable
                            colorscale='Viridis', # one of plotly colorscales
                            showscale=True)))
fig.add_trace(go.Scatter(x=result.index, 
                         y=result['ratio'],
                         name='Ratio',
                         mode='lines'))

fig.update_yaxes(type="log")

fig.update_layout(title='Stock-to-Flow Ratio',
                  showlegend=False,
                  width=1400, height=600)

fig.show()

In [ ]:
from coinmetrics.api_client import CoinMetricsClient

filename = 'data/on-chain-indicators.parquet'

table = pq.read_table(filename)
metrics = table.to_pandas()

last_date = metrics.index[-1].date()

if (last_date < date.today() - relativedelta(days=1)):
    # Obtain metrics by specifying the asset, date interval and frequency.
    # All metrics available: https://docs.coinmetrics.io/info/metrics
    client = CoinMetricsClient()
    metrics = client.get_asset_metrics(
        assets='btc'
        ,metrics=['PriceUSD',
                'AdrActCnt', #Addresses, active, count
                'AdrBalCnt', #Addresses, any balance, count
                'AdrBalNtv0.01Cnt', #Addresses, with balance, greater than 0.01 native units, count
                'AdrBalNtv0.1Cnt', #Addresses, with balance, greater than 0.1 native units, count
                'AdrBalNtv1Cnt', #Addresses, with balance, greater than 1 native units, count
                'AdrBalNtv10Cnt', #Addresses, with balance, greater than 10 native units, count
                'AdrBalNtv100Cnt', #Addresses, with balance, greater than 100 native units, count
                'AdrBalNtv1KCnt', #Addresses, with balance, greater than 1000 native units, count
                'AdrBalNtv10KCnt', #Addresses, with balance, greater than 10000 native units, count
                'CapMrktCurUSD', #Capitalization, market, current supply, USD
                'CapMVRVCur', #Capitalization, MVRV, current supply
                'NVTAdj', #NVT, adjusted
                'NVTAdj90', #NVT, adjusted, 90d MA
                'HashRate' #Hash rate, mean
                ]
        #,start_time='2010-07-10'
        #,end_time='2021-12-17'
        ,frequency='1d'
        )
    metrics = pd.DataFrame(metrics)
    metrics['time'] = pd.to_datetime(metrics['time'])
    metrics = metrics.set_index('time')
    metrics['PriceUSD'] = pd.to_numeric(metrics['PriceUSD'])
    metrics['AdrActCnt'] = pd.to_numeric(metrics['AdrActCnt'])
    metrics['AdrBalCnt'] = pd.to_numeric(metrics['AdrBalCnt'])
    metrics['AdrBalNtv0.01Cnt'] = pd.to_numeric(metrics['AdrBalNtv0.01Cnt'])
    metrics['AdrBalNtv0.1Cnt'] = pd.to_numeric(metrics['AdrBalNtv0.1Cnt'])
    metrics['AdrBalNtv1Cnt'] = pd.to_numeric(metrics['AdrBalNtv1Cnt'])
    metrics['AdrBalNtv10Cnt'] = pd.to_numeric(metrics['AdrBalNtv10Cnt'])
    metrics['AdrBalNtv100Cnt'] = pd.to_numeric(metrics['AdrBalNtv100Cnt'])
    metrics['AdrBalNtv1KCnt'] = pd.to_numeric(metrics['AdrBalNtv1KCnt'])
    metrics['AdrBalNtv10KCnt'] = pd.to_numeric(metrics['AdrBalNtv10KCnt'])
    metrics['CapMrktCurUSD'] = pd.to_numeric(metrics['CapMrktCurUSD'])
    metrics['CapMVRVCur'] = pd.to_numeric(metrics['CapMVRVCur'])
    metrics['NVTAdj'] = pd.to_numeric(metrics['NVTAdj'])
    metrics['NVTAdj90'] = pd.to_numeric(metrics['NVTAdj90'])
    metrics['HashRate'] = pd.to_numeric(metrics['HashRate'])

    table = pa.Table.from_pandas(metrics)
    pq.write_table(table, filename)

In [ ]:
metrics

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=metrics.index, y=metrics['PriceUSD'], name="Price"),
    secondary_y=False)

fig.add_trace(
    go.Scatter(x=metrics.index, y=metrics['NVTAdj90'], name="NVT"),
    secondary_y=True)

# Add figure title
fig.update_layout(
    title_text='BTC Price vs NVT')

# Set y-axes titles
fig.update_yaxes(title_text='Price', type='log', secondary_y=False)
fig.update_yaxes(title_text='NVT', secondary_y=True)

fig.show()

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=metrics.index, y=metrics['PriceUSD'], name="Price"),
    secondary_y=False)

fig.add_trace(
    go.Scatter(x=metrics.index, y=metrics['AdrBalCnt'], name="Number of Addresses"),
    secondary_y=True)

# Add figure title
fig.update_layout(
    title_text='Number of Addresses with a Non-Zero Balance')

# Set y-axes titles
fig.update_yaxes(title_text='Price', type='log', secondary_y=False)
fig.update_yaxes(title_text='Addresses with a Non-Zero Balance', secondary_y=True)

fig.show()

## Bitcoin Supply Distribution

We divide network entities according to their Bitcoin holdings into the following marine species:

* Shrimps (<1 BTC)
* Crab (1-10 BTC)
* Octopus (10-50 BTC)
* Fish (50-100 BTC)
* Dolphin (100-500 BTC)
* Shark (500-1,000 BTC)
* Whale (1,000-5,000 BTC)
* Humpback (>5,000 BTC).

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=metrics.index, 
                         y=metrics['AdrBalNtv0.01Cnt'],
                         name='>0.01',
                         mode='lines'))
fig.add_trace(go.Scatter(x=metrics.index, 
                         y=metrics['AdrBalNtv0.1Cnt'],
                         name='>0.1',
                         mode='lines'))
fig.add_trace(go.Scatter(x=metrics.index, 
                         y=metrics['AdrBalNtv1Cnt'],
                         name='>1',
                         mode='lines'))
fig.add_trace(go.Scatter(x=metrics.index, 
                         y=metrics['AdrBalNtv10Cnt'],
                         name='>10',
                         mode='lines'))
fig.add_trace(go.Scatter(x=metrics.index, 
                         y=metrics['AdrBalNtv100Cnt'],
                         name='>100',
                         mode='lines'))
fig.add_trace(go.Scatter(x=metrics.index, 
                         y=metrics['AdrBalNtv1KCnt'],
                         name='>1K',
                         mode='lines'))
fig.add_trace(go.Scatter(x=metrics.index, 
                         y=metrics['AdrBalNtv10KCnt'],
                         name='>10K',
                         mode='lines'))

fig.update_layout(title='Supply distribution',
                  width=1400, height=600)

fig.show()